In [1]:
from __future__ import annotations
import polars as pl
import seaborn as sns
import tqdm
from databento_dbn import FIXED_PRICE_SCALE, UNDEF_PRICE
# plot price of each over time
import plotly.graph_objs as go
import plotly.offline as pyo
import pandas as pd
import numpy as np
import datetime as dt  # For plotting x-axis as dates
import matplotlib.pyplot as plt

In [ ]:
import databento as db
import os 
current_dir = os.getcwd()

# Read all the files in the /data directory and store them in a list.
downloaded_files = [current_dir + '/data/' + file for file in os.listdir(current_dir + '/data')]
save_dir = "/Users/danny/trading/mfg/data/"
for file in sorted(downloaded_files):
    if file.endswith(".dbn.zst"):
        data = db.DBNStore.from_file(file)
        path = save_dir + file.split("/")[-1][:-8]
        data.to_df(price_type="fixed").to_csv(path + ".csv")

In [8]:
data = pl.read_parquet("/home/danny/hftbacktest/data/mbp.parquet")
data = data.filter((pl.col("symbol") == "GOOGL") | (pl.col("symbol") == "GOOG"))
data = data.filter(pl.col("price") != UNDEF_PRICE)
data = data.with_columns(pl.col("ts_event") - pl.col("ts_event").min())
data = data.filter(pl.col("price") < 200370000000.0)
data.shape

(7586, 75)

In [9]:
data.head()

ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,flags,ts_in_delta,sequence,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,bid_px_01,ask_px_01,bid_sz_01,ask_sz_01,bid_ct_01,ask_ct_01,bid_px_02,ask_px_02,bid_sz_02,ask_sz_02,bid_ct_02,ask_ct_02,bid_px_03,ask_px_03,bid_sz_03,ask_sz_03,bid_ct_03,ask_ct_03,bid_px_04,ask_px_04,bid_sz_04,ask_sz_04,bid_ct_04,ask_ct_04,bid_px_05,ask_px_05,bid_sz_05,ask_sz_05,bid_ct_05,ask_ct_05,bid_px_06,ask_px_06,bid_sz_06,ask_sz_06,bid_ct_06,ask_ct_06,bid_px_07,ask_px_07,bid_sz_07,ask_sz_07,bid_ct_07,ask_ct_07,bid_px_08,ask_px_08,bid_sz_08,ask_sz_08,bid_ct_08,ask_ct_08,bid_px_09,ask_px_09,bid_sz_09,ask_sz_09,bid_ct_09,ask_ct_09,symbol,__index_level_0__
"datetime[ns, UTC]",duration[ns],i64,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64
2024-01-31 13:00:56.891462406 UTC,0ns,10,41,7152,"""T""","""N""",0,143690000000,2,146,61218,58339,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,"""GOOGL""",4621
2024-01-31 13:00:57.775749154 UTC,884314878ns,10,41,7152,"""T""","""N""",0,143690000000,5,146,56140,58407,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,"""GOOGL""",4625
2024-01-31 13:01:16.622285997 UTC,19s 730825526ns,10,41,7152,"""T""","""N""",0,143690000000,1,146,66056,58974,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,"""GOOGL""",4669
2024-01-31 13:01:22.649014588 UTC,25s 757507757ns,10,41,7152,"""T""","""N""",0,143630000000,10,146,57247,59151,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,"""GOOGL""",4698
2024-01-31 13:01:25.224594870 UTC,28s 333010531ns,10,41,7151,"""T""","""N""",0,145290000000,11,146,77828,59263,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,9223372036854775807,9223372036854775

In [7]:
data["action"].unique()

action
str
"""A"""
"""C"""
"""T"""
"""M"""
